In [1]:
from xyz2graph import MolGraph, to_networkx_graph, to_plotly_figure
from plotly.offline import init_notebook_mode, iplot
import numpy as np
import pickle
# Initiate the Plotly notebook mode
from rdkit import Chem 

RDKit WARNING: [20:26:41] Enabling RDKit 2019.09.3 jupyter extensions


In [6]:
init_notebook_mode(connected=True)
mg = MolGraph()
mg.read_grofile('/root/repo/prepare/4G3E_2/SYSTEM/Protein/inter0~mol411/morph.gro')
f = to_plotly_figure(mg)
iplot(f)

In [5]:
file_name = '/root/repo/prepare/4G3E_2/SYSTEM/inter0_aligned_inter0~mol417.pickle'

mg = MolGraph()

with open(file_name, 'rb') as f:
    bundle = pickle.load(f)

mol1 = bundle['mol1']
mol2 = bundle['mol2']
mapping = bundle['mcs']

mg.draw_mol(mol1)
mg.draw_mol(mol2, pos_offset=10)
mg.generate_mapping(mapping)


# Create the Plotly figure object
f = to_plotly_figure(mg)
# f.layout.hovermode = 'closest'

adj_atoms = []
for atom, neighbors in mg.mapping_list.items():
    for neigh in neighbors:
        adj_atoms.append([atom, neigh])
        
def mapping_xyz(mg, adj_atoms):
    x = []
    y = []
    z = []
    for i, j in adj_atoms:
        x += (mg.x[i], mg.x[j], None)
        y += (mg.y[i], mg.y[j], None)
        z += (mg.z[i], mg.z[j], None)
    x = np.array(x).reshape(-1, 3)
    y = np.array(y).reshape(-1, 3)
    z = np.array(z).reshape(-1, 3)
    return x, y, z
    
    

def show_one_line(trace, points, selector):
    if points.point_inds:
        idx = points.point_inds[0]              
        mask = np.array([False] * len(adj_atoms))
        for i, pair  in enumerate(adj_atoms):
            if idx in pair:
                mask[i] = True

        new_x = x[mask, :].reshape(-1).tolist()
        new_y = y[mask, :].reshape(-1).tolist()
        new_z = z[mask, :].reshape(-1).tolist()


        if idx in mg.mapping_list:
            with f.batch_update(): 
                mapping_scatter['x'] = new_x
                mapping_scatter['y'] = new_y
                mapping_scatter['z'] = new_z

def show_no_line(trace, points, selector):
    print('leave')
    with f.batch_update():
        mapping_scatter['x'] = []
        mapping_scatter['y'] = []
        mapping_scatter['z'] = []
    

atom_scatter, bond_scatter, mapping_scatter = f.data

x, y, z = mapping_xyz(mg, adj_atoms)

atom_scatter.on_hover(show_one_line)                
# atom_scatter.on_unhover(show_no_line)
f

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'marker': {'color': [black, black, black, blac…